## Statistical Analysis of experimental results
First results are bootstrapped, then a Chi-square goodness of fit test is performed with the experimental results and global expectations

In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import heapq
from scipy.stats import chisquare

In [5]:
# import all necessary data from pickles dictionaries
V_dict = pickle.load(open("experiments_with_V/V_matrices.p", "rb"))

filename = 'VN_count_dict'
infile = open(filename,'rb')
VN_count_dict = pickle.load(infile)
infile.close()

filename = 'VN_count_std_dict'
infile = open(filename,'rb')
VN_count_std_dict = pickle.load(infile)
infile.close()

filename = 'VN_count_sample'
infile = open(filename,'rb')
VN_count_sample = pickle.load(infile)
infile.close()

filename = 'VN_count_sample_nodes'
infile = open(filename,'rb')
VN_count_sample_nodes = pickle.load(infile)
infile.close()

In [230]:
# set all parameters
V_list = [0, 1, 3, 4, 6, 9, 10, 11]
temperatures = [0.01, 0.5, 1, 1.5, 2.0, 2.5, 5, 10, np.inf] # np.arange(0.01)
samples = np.arange(0, 10, 1)
ts_arr = np.arange(0, 100000, 100)
max_possible_VN = [50, 28, 17, 18, 18, 10, 11, 10]
# max_possible_VN = [50, 31, 17, 18, 20, 9, 11, 7]

V_global_expectation = [1, 0.4444444444, 0.140625, 0.140625, 0.140625, 0.016, 0.016, 0.016]
V_global_counts = []
for i, v in enumerate(V_list):
    v_matrix = V_dict[f'{v}']
#     V_global_counts.append(V_global_expectation[i] * (100/v_matrix.shape[0]))
    V_global_counts.append(V_global_expectation[i] * max_possible_VN[i])

In [239]:
# bootstrap the results from the 10 samples per parameter setting
bootstrap_dict = {}
bootstrap_counts_dict = {}

for k, v in enumerate(V_list):
    print(k, v, max_possible_VN[k])
    v_matrix = V_dict[f'{v}']
    for temp in temperatures:
        df = pd.DataFrame(VN_count_dict[f'{v, temp, ts_arr[-1]}'])
        bootstrap = pd.DataFrame([df.sample(5, replace=True).mean()/(100/v_matrix.shape[0]) for i in range(1000)])
#         bootstrap = pd.DataFrame([df.sample(5, replace=True).mean()/(max_possible_VN[k]) for i in range(1000)])

        bootstrap_dict[f'{v, temp}'] = bootstrap
#         bootstrap_counts_dict[f'{v, temp}'] = bootstrap * (100/v_matrix.shape[0])
        bootstrap_counts_dict[f'{v, temp}'] = bootstrap * (max_possible_VN[k])
        #         bootstrap.hist(histtype='step')
        #         plt.axvline(bootstrap.mean()[0], color='C1')

0 0 50
1 1 28
2 3 17
3 4 18
4 6 18
5 9 10
6 10 11
7 11 10


In [240]:
data = pd.DataFrame(columns=V_list, index=temperatures)
data2 = pd.DataFrame(columns=V_list, index=temperatures)

for temp in temperatures:
    for v in V_list:
        data[v][temp] = bootstrap_counts_dict[f'{v, temp}'][0]
        data2[v][temp] = bootstrap_counts_dict[f'{v, temp}'][0].mean()

In [241]:
data

,0,1,3,4,6,9,10,11
0.01,0 50.0 1 50.0 2 50.0 3 50....,0 17.976 1 16.464 2 16.800 3 ...,0 6.800 1 6.664 2 6.800 3 ...,0 4.608 1 5.184 2 7.056 3 ...,0 12.960 1 11.664 2 12.960 3 ...,0 1.1 1 1.5 2 1.7 3 1.3 4 ...,0 0.99 1 0.44 2 0.44 3 0.5...,0 0.3 1 0.7 2 0.2 3 0.2 4 ...
0.50,0 50.0 1 50.0 2 50.0 3 50....,0 18.144 1 18.144 2 18.144 3 ...,0 7.480 1 7.344 2 7.072 3 ...,0 7.632 1 7.920 2 7.056 3 ...,0 11.088 1 11.808 2 11.808 3 ...,0 2.0 1 1.9 2 1.8 3 1.9 4 ...,0 0.44 1 1.43 2 0.55 3 1.5...,0 0.5 1 0.7 2 0.6 3 0.4 4 ...
1.00,0 48.4 1 48.4 2 48.0 3 48....,0 17.640 1 17.640 2 16.968 3 ...,0 6.120 1 5.576 2 6.800 3 ...,0 9.072 1 7.776 2 9.504 3 ...,0 11.376 1 11.952 2 12.384 3 ...,0 1.1 1 1.5 2 1.4 3 1.8 4 ...,0 2.09 1 2.53 2 3.08 3 3.4...,0 1.0 1 1.2 2 1.2 3 1.3 4 ...
1.50,0 46.2 1 47.8 2 45.2 3 44....,0 14.448 1 14.784 2 14.448 3 ...,0 4.624 1 4.488 2 4.352 3 ...,0 6.624 1 6.624 2 5.904 3 ...,0 7.920 1 8.352 2 6.768 3 ...,0 0.5 1 0.5 2 0.8 3 0.4 4 ...,0 1.43 1 1.21 2 2.09 3 1.8...,0 0.9 1 0.8 2 0.8 3 1.0 4 ...
2.00,0 44.2 1 45.4 2 44.8 3 46....,0 18.144 1 15.792 2 17.136 3 ...,0 5.168 1 4.488 2 6.664 3 ...,0 5.616 1 4.896 2 5.328 3 ...,0 6.336 1 5.760 2 6.912 3 ...,0 0.2 1 0.3 2 0.4 3 0.6 4 ...,0 1.32 1 1.65 2 1.54 3 2.0...,0 0.5 1 0.6 2 0.3 3 0.8 4 ...
2.50,0 46.8 1 46.0 2 44.0 3 45....,0 13.272 1 12.432 2 15.456 3 ...,0 3.536 1 5.440 2 5.168 3 ...,0 5.760 1 5.760 2 5.904 3 ...,0 5.328 1 5.040 2 5.760 3 ...,0 0.4 1 0.5 2 0.5 3 0.5 4 ...,0 1.98 1 1.21 2 1.32 3 1.4...,0 1.3 1 0.6 2 0.8 3 0.3 4 ...
5.00,0 44.2 1 44.4 2 42.2 3 42....,0 14.448 1 15.120 2 14.448 3 ...,0 3.264 1 4.080 2 2.992 3 ...,0 4.608 1 4.752 2 4.896 3 ...,0 4.320 1 4.320 2 4.464 3 ...,0 0.4 1 0.5 2 0.6 3 0.4 4 ...,0 1.43 1 0.55 2 0.99 3 0.7...,0 0.2 1 0.3 2 0.2 3 0.4 4 ...
10.00,0 43.0 1 43.2 2 43.0 3 43....,0 13.608 1 14.616 2 12.432 3 ...,0 2.720 1 3.264 2 2.584 3 ...,0 3.744 1 4.608 2 4.176 3 ...,0 3.312 1 4.032 2 2.880 3 ...,0 0.2 1 0.2 2 0.2 3 0.3 4 ...,0 0.88 1 0.88 2 0.88 3 0.6...,0 0.1 1 0.4 2 0.2 3 0.0 4 ...
inf,0 42.0 1 43.6 2 41.8 3 43....,0 15.456 1 13.944 2 14.112 3 ...,0 2.856 1 4.216 2 3.400 3 ...,0 3.744 1 3.888 2 4.032 3 ...,0 3.312 1 3.168 2 3.168 3 ...,0 0.0 1 0.3 2 0.3 3 0.2 4 ...,0 0.66 1 0.55 2 0.66 3 0.2...,0 0.1 1 0.0 2 0.0 3 0.0 4 ...


In [242]:
data2

,0,1,3,4,6,9,10,11
0.01,50.0,18.03396,6.798776,5.88168,12.391056,1.4479,0.55506,0.308
0.50,50.0,17.50644,7.250976,7.551792,12.850128,1.7444,1.47917,0.6031
1.00,47.896,17.4048,6.451432,8.785296,11.868912,1.5004,2.70523,1.0879
1.50,46.2088,14.90748,4.407624,6.533856,7.545888,0.4591,1.41702,0.9519
2.00,45.2286,16.283568,4.993784,5.791824,6.319872,0.4066,1.31813,0.5426
2.50,45.2992,14.26236,4.206752,5.597136,5.245056,0.3997,1.30966,0.594
5.00,43.4692,13.693848,3.146904,4.692816,4.50864,0.4481,0.97812,0.2972
10.00,43.0776,13.91796,3.008728,3.979584,3.470256,0.3363,0.71577,0.2587
inf,43.1208,13.614888,3.327376,4.181472,3.117888,0.1949,0.48499,0.1066


## $\chi^2$ -  Goodness of Fit Test

H$_0$: The samples follow the hypothesized distribution. If P > 0.05 H$_0$ cannot be rejected.

H$_A$: The samples do not follow the hypothesized distribution. If p < 0.05 H$_0$ can be rejected

In [243]:
# per temperature, for all value networks
for temp in temperatures:
    print('temperature:', temp)
    chi2 = chisquare(data2[:].loc[temp], f_exp=V_global_counts)
    if chi2[1] > 0.05:
        print('accept H0, expectations are similar, p-value =', chi2[1])
    else:
        print('reject H0, expectations are off, p-value =', chi2[1])

temperature: 0.01
reject H0, expectations are off, p-value = 4.214942212431874e-05
temperature: 0.5
reject H0, expectations are off, p-value = 1.5877582054865455e-07
temperature: 1
reject H0, expectations are off, p-value = 1.6973693869591776e-09
temperature: 1.5
accept H0, expectations are similar, p-value = 0.07726686726403163
temperature: 2.0
accept H0, expectations are similar, p-value = 0.31320742132640844
temperature: 2.5
accept H0, expectations are similar, p-value = 0.5401284922404141
temperature: 5
accept H0, expectations are similar, p-value = 0.8796241211677706
temperature: 10
accept H0, expectations are similar, p-value = 0.9766726431796565
temperature: inf
accept H0, expectations are similar, p-value = 0.9822705843443955


In [244]:
# per value network, for all temperatures
for i, v in enumerate(V_list):
    chi2 = chisquare(data2[v].loc[:], f_exp=V_global_counts[i])
#     print(data2[v].loc[:],'\n count', V_global_counts[i])
    print("VN", v)
    if chi2[1] > 0.1:
        print('accept H0, expectations are similar, p-value =', chi2[1])
    else:
        print('reject H0, expectations are off, p-value =', chi2[1])

VN 0
accept H0, expectations are similar, p-value = 0.8543009202590978
VN 1
accept H0, expectations are similar, p-value = 0.9796200891554472
VN 3
accept H0, expectations are similar, p-value = 0.22590540882691781
VN 4
reject H0, expectations are off, p-value = 0.010321349872511675
VN 6
reject H0, expectations are off, p-value = 4.769888401660983e-13
VN 9
reject H0, expectations are off, p-value = 0.06162561844267743
VN 10
reject H0, expectations are off, p-value = 4.039570383307766e-05
VN 11
accept H0, expectations are similar, p-value = 0.8672099374601239


In [245]:
# all value network, all temperatures
import math

chi2_matrix = pd.DataFrame(columns=V_list, index=temperatures)
chi2_matrix = np.zeros((len(temperatures), len(V_list)))
chi2_text = pd.DataFrame(columns=V_list, index=temperatures)

for i, v in enumerate(V_list):
    for j, temp in enumerate(temperatures):
#         print(i, v, temp, V_global_counts[i], data[v].loc[temp])
        chi2 = chisquare(data[v].loc[temp], f_exp=[V_global_counts[i] for x in data[v].loc[temp]])
        chi2_matrix[j][i] = chi2[1]
        
        if chi2[1] > 0.01:
            print('accept H0, expectations are similar, p-value =', chi2[1])
            chi2_text[v][temp] = '*'
        else:
            print('reject H0, expectations are off, p-value =', chi2[1])
            chi2_text[v][temp] = '-'
            
print(chi2_matrix)


accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 0.9989906290162138
accept H0, expectations are similar, p-value = 0.7887332544387535
accept H0, expectations are similar, p-value = 0.7996549892129269
accept H0, expectations are similar, p-value = 0.9999999999553055
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
accept H0, expectations are similar, p-value = 1.0
reject H0, expectation

In [246]:
import numpy as np 
from pandas import DataFrame
import seaborn as sns
from matplotlib import cm
%matplotlib inline

df = DataFrame(chi2_matrix, index=temperatures, columns=V_list)

f, ax = plt.subplots(figsize=(10, 10), dpi=1000)
ax = sns.heatmap(df, annot=True, cmap=cm.coolwarm, linewidths=.5)

# ax.set_yticks(np.arange(len(temperatures)))
yl = np.array([str(i) for i in temperatures], dtype = object)
yl[-1] = r"$\infty$"
ax.set_yticklabels(yl)
ax.set_ylabel('Temperatures')

# ax.set_xticks(np.arange(len(V_list)))
xl = np.array([str(i) for i in V_list], dtype = object)
ax.set_xticklabels(xl)
ax.set_xlabel('Value Network')

plt.title(r"P-Values $\chi^2$ Goodness of Fit Test")
plt.savefig('2d_pvalue.png', dpi = 1000)

## $\chi^2$ -  Test for Interdependence

This test shows how the multiple value networks and temperatures are independent of each other.

In [247]:
import scipy.stats as stats 
# stats.chi2_contingency(data2)
data3 = stats.chi2_contingency(data2)[3]

pd.DataFrame(
    data=data3[:,:], 
    index=temperatures,
    columns=V_list
).round(2)

,0,1,3,4,6,9,10,11
0.01,53.39,17.99,5.62,6.83,8.67,0.89,1.41,0.61
0.50,55.38,18.66,5.83,7.08,9.00,0.93,1.47,0.63
1.00,54.66,18.42,5.75,6.99,8.88,0.92,1.45,0.63
1.50,46.12,15.54,4.85,5.90,7.49,0.77,1.22,0.53
2.00,45.26,15.25,4.76,5.79,7.35,0.76,1.20,0.52
2.50,43.03,14.50,4.53,5.50,6.99,0.72,1.14,0.49
5.00,39.86,13.43,4.19,5.10,6.48,0.67,1.05,0.46
10.00,38.47,12.97,4.05,4.92,6.25,0.64,1.02,0.44
inf,38.13,12.85,4.01,4.88,6.20,0.64,1.01,0.44


In [248]:
chi, pval, dof, exp = stats.chi2_contingency(data2)
print('p-value is: ', pval)
significance = 0.05
p = 1 - significance
critical_value = stats.chi2.ppf(p, dof)
print('chi=%.6f, critical value=%.6f\n' % (chi, critical_value))
if chi > critical_value:
    print("""At %.2f level of significance, we reject the null hypotheses and accept H1. 
The samples not independent.""" % (significance))
else:
    print("""At %.2f level of significance, we accept the null hypotheses. 
The samples are independent.""" % (significance))

p-value is:  0.9999980306198529
chi=19.855095, critical value=74.468324

At 0.05 level of significance, we accept the null hypotheses. 
The samples are independent.
